In [1]:
library(rhdf5)
library(MASS)

In [2]:
# read in previously published enhancer-gene pairs for validation
enhancer.gene.pairs <- read.csv('/iblm/netapp/data1/external/Gasperini2019/gasperini_enhancer_gene_pairs_suppl_table_2.csv')
head(enhancer.gene.pairs)

,Target_Site,ENSG,target_gene_short,Diff_expression_test_raw_pval,Diff_expression_test_fold_change,Diff_expression_test_Empirical_pval,Diff_expression_test_Empirical_adjusted_pval,high_confidence_subset,chr.candidate_enhancer,start.candidate_enhancer,stop.candidate_enhancer
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<int>,<int>
1,chr2.2482,ENSG00000115977,AAK1,1.451572e-03,0.7565418,0.002719115,0.09865186,TRUE,chr2,69056234,69056865
2,chrX.2695,ENSG00000101986,ABCD1,7.351840e-04,0.6693686,0.001824566,0.07301369,TRUE,chrX,153250743,153251468
3,chr10.2252,ENSG00000138316,ADAMTS14,2.140000e-09,0.4473551,0.000449245,0.03249770,FALSE,chr10,72426863,72427518
4,chr1.8461,ENSG00000143382,ADAMTSL4,1.157260e-04,0.6676576,0.001044298,0.05041364,TRUE,chr1,150517877,150518596
5,chr11.1006,ENSG00000148926,ADM,5.881130e-04,0.3091749,0.001651173,0.06733578,TRUE,chr11,9573345,9573973
6,chr2.7130,ENSG00000157985,AGAP1,2.390000e-07,0.5843086,0.000583231,0.03626832,TRUE,chr2,236369078,236369902


In [3]:
# define sample target site and sample target gene
sample.target.site <- 'chr2.2482'
sample.target.gene <- 'ENSG00000115977'

In [4]:
# read in table that connects target sites to guide spacer sequences
target.site.spacers.table <- read.table(
    '/iblm/netapp/data1/external/Gasperini2019/suppl/GSE120861_grna_groups.at_scale.txt',
    sep = '\t'
)
colnames(target.site.spacers.table) <- c('target.site', 'spacer.sequence')
target.site.spacers.table$target.site <- sapply(target.site.spacers.table$target.site, FUN = function(x) {
    if (startsWith(x, 'chr')) {
        return (strsplit(x, '_')[[1]][1])
    }
    else {
        return (x)
    }
})
head(target.site.spacers.table)

,target.site,spacer.sequence
,<chr>,<chr>
1,SH3BGRL3_TSS,AAACCGCTCCCGAGCACGGG
2,MTRNR2L8_TSS,AAATAGTGGGAAGATTCGTG
3,FAM83A_TSS,AACACACCACGGAGGAGTGG
4,ZNF593_TSS,AACAGCCCGGCCGGCCAAGG
5,ATPIF1_TSS,AACGAGAGACTGCTTGCTGG
6,TIPRL_TSS,AACGGCTCGGAAGCCTAGGG


In [5]:
# get spacer sequences of guides coresponding to target site
target.site.spacers <- target.site.spacers.table[target.site.spacers.table$target.site == sample.target.site, ]$spacer.sequence
target.site.spacers

[1] "GCTGCTATCTGGAAAGGAAG" "TGCCACCCTGAGCTTCTGAG"

In [7]:
# read in guide efficiency information
all.guide.efficiencies <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'guidescan.output')
all.guide.efficiencies$spacer <- substring(all.guide.efficiencies$gRNA, 1, nchar(all.guide.efficiencies$gRNA) - 3)
head(all.guide.efficiencies)

,Index,gRNA,Chromosome,Start,End,Strand,Num.Off.Targets,Off.Target.Summary,Specificity,Cutting.Efficiency,spacer
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<dbl>,<chr>
1,0,CTAAAGCATTGGCTGAGAAGNGG,chr8,23911081,23911103,-,41,2:2 | 3:39,0.13622800,0.565010,CTAAAGCATTGGCTGAGAAG
2,1,GTAGTTCACATAATCCCTGTNGG,chr4,25698193,25698215,-,55,2:0 | 3:55,0.16592900,0.572492,GTAGTTCACATAATCCCTGT
3,2,AAGTTGACTCTACATAGCAGNGG,chr8,23912565,23912587,+,22,2:1 | 3:21,0.34106700,0.636691,AAGTTGACTCTACATAGCAG
4,3,AATATTCTCCCTCATTCTGGNGG,chr5,12539360,12539382,-,803,2:26 | 3:777,0.00274364,0.619800,AATATTCTCCCTCATTCTGG
5,4,AATCCTCTAATGGACGAAGANGG,chr8,23913057,23913079,-,24,2:0 | 3:24,0.33441500,0.602272,AATCCTCTAATGGACGAAGA
6,5,AGATACCTATGGCCATATAGNGG,chr5,12540099,12540121,+,14,2:0 | 3:14,0.35172300,0.531946,AGATACCTATGGCCATATAG


In [9]:
target.site.spacers.efficiencies <- all.guide.efficiencies[all.guide.efficiencies$spacer %in% target.site.spacers, c('spacer', 'Cutting.Efficiency')]
target.site.spacers.efficiencies

,spacer,Cutting.Efficiency
,<chr>,<dbl>
4933,GCTGCTATCTGGAAAGGAAG,0.608583
4934,TGCCACCCTGAGCTTCTGAG,0.683395


In [10]:
# read in cell-guide matrix
cell.guide.matrix <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'cell.guide.matrix')
guide.spacers <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'guide.spacers')
print(head(cell.guide.matrix))
print(head(guide.spacers))

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    1    1    1    1    1    1    1     1     1     1     1     1
[2,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
[3,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
[4,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
[5,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
[6,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     1     1     1     1     1     1     1     1     1     1
[2,]     0     0     0     0     0     0     0     0     0     0     0     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
[4,]     0     0     0     0     0     0     0     0     0     0     0     0
[5,]     0     0     0     0     0     0     0     0   

In [11]:
colnames(cell.guide.matrix) <- guide.spacers

In [14]:
guide1.indicator.vector <- cell.guide.matrix[, target.site.spacers.efficiencies$spacer[1]]
guide2.indicator.vector <- cell.guide.matrix[, target.site.spacers.efficiencies$spacer[2]]
guide1.efficiency <- target.site.spacers.efficiencies$Cutting.Efficiency[1]
guide1.efficiency <- target.site.spacers.efficiencies$Cutting.Efficiency[2]

In [20]:
# compute indicator vector
combined.indicator.vector <- guide1.indicator.vector + guide2.indicator.vector
for (i in 1:length(combined.indicator.vector)) {

    if (combined.indicator.vector[i] == 2) {
        return (rbern(1, 1 - (1 - guide1.efficiency) * (1 - guide2.efficiency)))
    }

    if (combined.indicator.vector[i] == 1) {

        if (guide1.indicator.vector[i] == 1) {
            return (rbern(1, guide1.efficiency))
        }

        if (guide2.indicator.vector[i] == 1) {
            return (rbern(1, guide2.efficiency))
        }
    }

    if (combined.indicator.vector[i] == 0) {
        return (0)
    }

    return (0)
}
combined.indicator.vector

[1] 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   [37] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   [73] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0
  [109] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [145] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [181] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [217] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [253] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [289] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [325] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [361] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [397] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [433] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [469] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [505] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [541] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [577] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [613] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [649] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [685] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [721] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [757] 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [793] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [829] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [865] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [901] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [937] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [973] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 [1009] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1045] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1081] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1117] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1153] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1189] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1225] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1261] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1297] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1333] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1369] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1405] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1441] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1477] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 [1513] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1549] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1585] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 [1621] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1657] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1693] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1729] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1765] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [18

In [21]:
# read in counts matrix
gene.counts <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'gene.counts')
gene.names <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'gene.names')
rownames(gene.counts) <- gene.names
head(gene.counts)

ENSG00000238009,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000237683,1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,1,0,0,0,1,0,0
ENSG00000228463,1,0,1,1,1,0,0,0,0,1,⋯,0,0,0,0,1,0,0,2,0,2
ENSG00000237094,0,0,0,1,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000235373,0,0,1,0,0,0,0,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000228327,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [23]:
target.gene.counts <- gene.counts[sample.target.gene, ]

In [35]:
scaling.factors <- colSums(gene.counts) / sum(colSums(gene.counts))
head(scaling.factors)

[1] 4.662730e-06 2.366934e-06 3.882334e-06 6.047533e-06 2.687321e-06
[6] 2.586188e-06

In [36]:
length(scaling.factors)

[1] 207324

In [24]:
# read in covariates
covariates <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'covariates')
cell.barcodes <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'cell.barcodes')
head(covariates)


,cell,guide_count,prep_batch,percent.mito,s.score,g2m.score
,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>
1,AAACCTGAGAACTGTA-1_2A_4_SI-GA-G5,24,prep_batch_2,0.05442845,0.05704670,0.002865079
2,AAACCTGAGAAGAAGC-1_2B_5_SI-GA-H6,30,prep_batch_2,0.06170906,-0.05632492,0.225004169
3,AAACCTGAGAAGGTTT-1_2A_7_SI-GA-G8,18,prep_batch_2,0.06625193,-0.17070798,0.167475723
4,AAACCTGAGAATAGGG-1_2B_6_SI-GA-H7,34,prep_batch_2,0.01928651,-0.10711479,1.108284663
5,AAACCTGAGAATTCCC-1_1B_2_SI-GA-F3,13,prep_batch_1,0.03646176,-0.15393309,-0.145533466
6,AAACCTGAGACACTAA-1_1B_5_SI-GA-F6,NA,prep_batch_1,0.03699082,-0.05339071,-0.188043139


In [26]:
head(cell.barcodes)

[1] "AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2" "AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2"
[3] "AAACCTGCAAACAACA-1_1A_1_SI-GA-E2" "AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2"
[5] "AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2" "AAACCTGGTAGCGCAA-1_1A_1_SI-GA-E2"

In [29]:
# reorder covariates to match indicator and count vectors
covariates <- merge(data.frame(cell.barcodes), covariates, by.x = 'cell.barcodes', by.y = 'cell', sort = FALSE)
head(covariates)

,cell.barcodes,guide_count,prep_batch,percent.mito,s.score,g2m.score
,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>
1,AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2,67,prep_batch_1,0.058786706,0.110732311,-0.1319208
2,AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2,26,prep_batch_1,0.036086518,-0.010290919,-0.1535426
3,AAACCTGCAAACAACA-1_1A_1_SI-GA-E2,61,prep_batch_1,0.069823051,-0.175860130,-0.3084879
4,AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2,39,prep_batch_1,0.026186508,0.003057281,-0.1574859
5,AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2,37,prep_batch_1,0.007991318,-0.144480961,-0.2362154
6,AAACCTGGTAGCGCAA-1_1A_1_SI-GA-E2,57,prep_batch_1,0.022356681,0.026418076,-0.1462899


In [32]:
# create merged table with counts, indicator vector, and covariates
enhancer.gene.model.df <- cbind(covariates, combined.indicator.vector, target.gene.counts)
head(enhancer.gene.model.df)

,cell.barcodes,guide_count,prep_batch,percent.mito,s.score,g2m.score,combined.indicator.vector,target.gene.counts
,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2,67,prep_batch_1,0.058786706,0.110732311,-0.1319208,0,0
2,AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2,26,prep_batch_1,0.036086518,-0.010290919,-0.1535426,0,0
3,AAACCTGCAAACAACA-1_1A_1_SI-GA-E2,61,prep_batch_1,0.069823051,-0.175860130,-0.3084879,0,1
4,AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2,39,prep_batch_1,0.026186508,0.003057281,-0.1574859,0,0
5,AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2,37,prep_batch_1,0.007991318,-0.144480961,-0.2362154,0,0
6,AAACCTGGTAGCGCAA-1_1A_1_SI-GA-E2,57,prep_batch_1,0.022356681,0.026418076,-0.1462899,1,0


In [41]:
model <- glm.nb(
    formula = target.gene.counts ~ combined.indicator.vector + prep_batch + guide_count + percent.mito + s.score + g2m.score + offset(scaling.factors),
    data = enhancer.gene.model.df
)

In [42]:
summary(model)


Call:
glm.nb(formula = target.gene.counts ~ combined.indicator.vector + 
    prep_batch + guide_count + percent.mito + s.score + g2m.score + 
    offset(scaling.factors), data = enhancer.gene.model.df, init.theta = 0.6779811244, 
    link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.9774  -0.5407  -0.4762  -0.4113   5.8167  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -2.4910563  0.0210222 -118.50  < 2e-16 ***
combined.indicator.vector -0.3004809  0.0918987   -3.27  0.00108 ** 
prep_batchprep_batch_2    -0.0414817  0.0129628   -3.20  0.00137 ** 
guide_count                0.0115322  0.0004054   28.45  < 2e-16 ***
percent.mito               3.7925975  0.3641216   10.42  < 2e-16 ***
s.score                   -2.8525064  0.0623904  -45.72  < 2e-16 ***
g2m.score                 -0.3557967  0.0335960  -10.59  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dis